# SimCLR Tree Segmentation Training and Evaluation

In [ ]:
pip install torchmetrics

In [ ]:
pip install pytorch-metric-learning

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from datetime import datetime
from pytorch_metric_learning.losses import NTXentLoss

from config import device, data_config
from data.dataloaders import create_dataloaders
from models.encoders import ResNet50Encoder
from models.simclr import SimCLRModel, ProjectionHead
from models.segmentation import TreeSegmentationModel, BaselineTreeSegmentationModel
from training.simclr_trainer import train_simclr
from training.segmentation_trainer import train_segmentation, evaluate_segmentation
from training.baseline_trainer import train_baseline_segmentation, evaluate_baseline_segmentation
from utils.visualization import plot_losses_comparison, visualize_segmentation_results_colored
from utils.io_utils import save_comparison_data

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Training run timestamp: {timestamp}")

print("Creating data loaders...")
pretrain_loader, seg_train_loader, seg_val_loader = create_dataloaders(**data_config)

# Optionally visualize a batch
batch = next(iter(seg_train_loader))
images, masks = batch[:2]
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.subplot(1,2,1), plt.imshow(images[0].permute(1,2,0)), plt.title('Image')
plt.subplot(1,2,2), plt.imshow(masks[0][0]), plt.title('Mask')
plt.show()

In [ ]:
baseline_model = BaselineTreeSegmentationModel(num_classes=2).to(device)
baseline_optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-4, weight_decay=1e-5)
baseline_criterion = nn.CrossEntropyLoss()

baseline_train_losses, baseline_val_losses = train_baseline_segmentation(
    baseline_model, seg_train_loader, seg_val_loader,
    baseline_optimizer, baseline_criterion, device, epochs=15, patience=5
)

if seg_val_loader is not None:
    print("Final evaluation on validation set (Baseline)")
    baseline_val_loss, baseline_pixel_acc, baseline_iou_score, baseline_dice_score = evaluate_baseline_segmentation(
        baseline_model, seg_val_loader, device
    )

In [ ]:
encoder = ResNet50Encoder(hidden_dim=512)
projection_head = ProjectionHead(input_dim=512, hidden_dim=512, output_dim=128)
simclr_model = SimCLRModel(encoder, projection_head).to(device)

optimizer = torch.optim.Adam(simclr_model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = NTXentLoss(temperature=0.5)
simclr_losses = train_simclr(simclr_model, pretrain_loader, optimizer, criterion, device, epochs=50)

In [ ]:
seg_model = TreeSegmentationModel(simclr_model.encoder, num_classes=2).to(device)
seg_optimizer = torch.optim.Adam(seg_model.parameters(), lr=1e-4, weight_decay=1e-5)
seg_criterion = nn.CrossEntropyLoss()
seg_train_losses, seg_val_losses = train_segmentation(
    seg_model, seg_train_loader, seg_val_loader,
    seg_optimizer, seg_criterion, device, epochs=50, patience=5
)

if seg_val_loader is not None:
    final_val_loss, final_pixel_acc, final_iou_score, final_dice_score = evaluate_segmentation(
        seg_model, seg_val_loader, device
    )

In [ ]:

print("\nBaseline Model Results:")
print(f"  Final train loss: {baseline_train_losses[-1]:.4f}")
if baseline_val_losses:
    print(f"  Final val loss: {baseline_val_losses[-1]:.4f}")
if seg_val_loader is not None:
    print(f"  Final pixel accuracy: {baseline_pixel_acc:.4f}")
    print(f"  Final IoU score: {baseline_iou_score:.4f}")
    print(f"  Final Dice score: {baseline_dice_score:.4f}")

print("\nSimCLR Pretrained Model Results:")
print(f"  SimCLR final loss: {simclr_losses[-1]:.4f}")
print(f"  Segmentation final train loss: {seg_train_losses[-1]:.4f}")
if seg_val_losses:
    print(f"  Segmentation final val loss: {seg_val_losses[-1]:.4f}")
if seg_val_loader is not None:
    print(f"  Final pixel accuracy: {final_pixel_acc:.4f}")
    print(f"  Final IoU score: {final_iou_score:.4f}")
    print(f"  Final Dice score: {final_dice_score:.4f}")

if seg_val_loader is not None:
    iou_improvement = ((final_iou_score - baseline_iou_score) / baseline_iou_score) * 100
    dice_improvement = ((final_dice_score - baseline_dice_score) / baseline_dice_score) * 100
    acc_improvement = ((final_pixel_acc - baseline_pixel_acc) / baseline_pixel_acc) * 100
    print(f"\nImprovement with SimCLR pretraining:")
    print(f"  IoU improvement: {iou_improvement:+.2f}%")
    print(f"  Dice improvement: {dice_improvement:+.2f}%")
    print(f"  Pixel accuracy improvement: {acc_improvement:+.2f}%")

# Visualize loss comparison if you have a matplotlib plot function
plot_filename = plot_losses_comparison(
    simclr_losses, seg_train_losses, baseline_train_losses, timestamp,
    baseline_iou_score, baseline_dice_score, baseline_pixel_acc,
    final_iou_score, final_dice_score, final_pixel_acc
)
from IPython.display import Image
display(Image(plot_filename))

In [ ]:
visualize_segmentation_results_colored(seg_model, seg_val_loader, device, num_images=7)

In [ ]:
baseline_model_path = f'baseline_tree_segmentation_model_{timestamp}.pth'
simclr_model_path = f'simclr_pretrained_resnet50_{timestamp}.pth'
seg_model_path = f'tree_segmentation_model_resnet50_{timestamp}.pth'
torch.save(baseline_model.state_dict(), baseline_model_path)
torch.save(simclr_model.state_dict(), simclr_model_path)
torch.save(seg_model.state_dict(), seg_model_path)

print(f"\nModels saved:")
print(f"  Baseline: {baseline_model_path}")
print(f"  SimCLR: {simclr_model_path}")
print(f"  Segmentation: {seg_model_path}")